利用vgg提取每一类的平均的特征，保存特征

In [1]:
import os
import glob
import json
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
#from progressbar import * #进度条
from tqdm import tqdm #进度条
from torchnet import meter
from natsort import natsorted
import matplotlib.pyplot as plt
import torch.nn.functional as F #torch是关于运算的包
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms as T
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集
os.environ['CUDA_VISIBLE_DEVICES']='2'

### Vgg模型 

#### 再次训练

In [2]:
train_csv = pd.read_csv("data/train.csv")
temp_df = train_csv.groupby('Id').count()
#temp_df_subset = temp_df[temp_df.Image == 1]
temp_df_subset = temp_df[temp_df.Image >= 30]
#print(temp_df_subset)

In [3]:
traindata = torchvision.datasets.ImageFolder('data/net_classes',
                                            transform = transforms.Compose([
                                                transforms.Grayscale(3),
                                                transforms.Resize((224, 224)),
                                                T.RandomHorizontalFlip(),
                                                transforms.ToTensor()
                                            ]))

In [4]:
print(traindata[0][0].shape)
print(traindata[454][1])#打印标签
print('train len is %d' % len(traindata))

torch.Size([3, 224, 224])
10
train len is 1325


In [5]:
pre_model = models.vgg16(pretrained=True)
for parma in pre_model.parameters():
    parma.requires_grad = False
pre_model.classifier = torch.nn.Sequential(torch.nn.Linear(25088, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 2048),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(2048, 31))

In [6]:
#print(pre_model)
net = pre_model
y = net(torch.randn(1, 3, 224, 224))
#print(net)
print(y.size())

torch.Size([1, 31])


In [7]:
def train(model_path, train_dataset):
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
    net = pre_model
    #接着上一次训练
    if os.path.exists(model_path):
        net = torch.load(model_path)
    net = net.cuda()
    #调整学习率参数
    lr = 0.001
    lr_decay = 0.995
    weight_decay = 1e-4
    pre_train_loss = 100000
    
    # CrossEntropyLoss就是我们需要的损失函数
    criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
    #统计指标
    #loss_meter = meter.AverageValueMeter()
    #保存模型判断条件
    max_train_acc = 0
    pre_epoch = 0
    max_interval_epoch = 10

    
    print("Start Training...")
    for epoch in range(100):
        #训练集
        train_loss = 0
        train_count = 0
        train_acc = 0
        net.train()
        for i, data in enumerate(trainloader):
            inputs, labels = data
            #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            #更新指标
            train_count += labels.size(0)
            train_loss += loss.item()
            _, predict = outputs.max(1)
            train_acc += (predict == labels).sum().item()
        train_loss /= train_count
        train_acc /= train_count
        
        print('the epoch %d, the train loss is %f, the trian acc is %f' % (epoch, train_loss, train_acc))
        
        #保存模型
        if train_acc > max_train_acc:
            max_train_acc = train_acc
            pre_epoch = epoch
            torch.save(net, model_path)#保存整个神经网络的的结构信息和模型参数信息，save的对象是网络net
        if epoch - pre_epoch > max_interval_epoch:
            print('early stop')
            break
            
        #如果损失不载下降，则降低学习率
        if train_loss > pre_train_loss:
            lr = lr*lr_decay
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        pre_train_loss = pre_train_loss
    print("Done Training!")

In [8]:
train('model/funtune_vgg16.pkl', traindata)

Start Training...
the epoch 0, the train loss is 0.023060, the trian acc is 0.174340
the epoch 1, the train loss is 0.022088, the trian acc is 0.204528
the epoch 2, the train loss is 0.021750, the trian acc is 0.221132
the epoch 3, the train loss is 0.020545, the trian acc is 0.264151
the epoch 4, the train loss is 0.018870, the trian acc is 0.313962
the epoch 5, the train loss is 0.016643, the trian acc is 0.396226
the epoch 6, the train loss is 0.013860, the trian acc is 0.487547
the epoch 7, the train loss is 0.010813, the trian acc is 0.598491
the epoch 8, the train loss is 0.008971, the trian acc is 0.669434
the epoch 9, the train loss is 0.007110, the trian acc is 0.732075
the epoch 10, the train loss is 0.006102, the trian acc is 0.763774
the epoch 11, the train loss is 0.005536, the trian acc is 0.784906
the epoch 12, the train loss is 0.004770, the trian acc is 0.812075
the epoch 13, the train loss is 0.003989, the trian acc is 0.855094
the epoch 14, the train loss is 0.003835

In [8]:
print(*list(pre_model.classifier.children())[:-3])
#print(pre_model.features)
#vgg_model = models.vgg16(pretrained=True)
#print(vgg_model)

Linear(in_features=25088, out_features=4096, bias=True) ReLU() Dropout(p=0.5) Linear(in_features=4096, out_features=2048, bias=True)


In [9]:
#vgg_model = models.vgg16(pretrained=True)
#print(vgg_model)
class Encoder(nn.Module):
    '''
    提取vgg的4096向量
    '''
    def __init__(self, model_path):
        super(Encoder, self).__init__()
        VGG = torch.load(model_path)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output

In [11]:
#print(pre_model)
model = Encoder('model/funtune_vgg16.pkl')
model = model.cuda()
x = torch.randn(1, 3, 224, 224)
x = x.cuda()
y = model(x)
#print(net)
print(y.size())

torch.Size([1, 2048])


In [20]:
#print(Encoder())
def extractor(img_path, net):
    transform = transforms.Compose([
        transforms.Grayscale(3),
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    img = Image.open(img_path)
    img = transform(img)
    img = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)#增加一维
    #print(img.shape)
    net = net.cuda()
    net.eval()
    img = img.cuda()
    y = net(img).cpu()
    y = torch.squeeze(y)
    y = y.data.numpy()
    
    return y

In [21]:
img_path = 'data/train/1cf255a78.jpg'
model_path = 'model/funtune_vgg16.pkl'
feature = extractor(img_path, Encoder(model_path))
#feature = extractor(img_path, Encoder())
print(feature)

[-0.00475063  0.23318964  0.7165816  ... -0.6850112  -1.4086967
  0.37565774]


### 提取特征

In [22]:
def extract_train_feature(dir_path, model, feature_dim=2048):
    data = np.zeros((1, feature_dim))
    label = []
    for file in tqdm(os.listdir(dir_path)):
        if file == 'new_whale':
            continue
        path = os.path.join(dir_path, file)
        if os.path.isdir(path):
            feature_tmp = np.zeros((1, feature_dim))
            label_name = file
            img_num = 0
            for img in os.listdir(path):
                img_path = os.path.join(path, img)
                feature = extractor(img_path, model)
                img_num += 1
                feature_tmp += feature
            if img_num > 0:
                feature = feature_tmp / img_num
                data = np.concatenate((data, feature))
                label.append(label_name)
    data = data[1:, :]
    np.savetxt('model/train_feature.txt', data, fmt='%f')
    label_file = open('model/label.txt', 'w')
    json.dump(label, label_file)
    label_file.close

In [23]:
extract_train_feature('data/train_classes',  Encoder(model_path))

100%|██████████| 5005/5005 [19:54<00:00,  6.59it/s]  


### 找到最近的5个类别

In [24]:
train_features = np.loadtxt('model/train_feature.txt', dtype=float)
train_labels = open('model/label.txt', 'r')
labels = json.load(train_labels)
train_labels.close()
#print(train_features)
#print(labels)

In [25]:
def top_5_classes(test_feature, train_features, labels):
    #print(train_features.shape)
    distance = train_features - test_feature
    euclidean_distance = np.linalg.norm(distance, axis=1, keepdims=True)
    euclidean_distance = euclidean_distance.flatten()
    min_distance_idx = np.argsort(euclidean_distance)[:5]
    min_distance = np.sort(euclidean_distance)[:5]
    #print(min_distance)
    classes = []
    for idx in min_distance_idx:
        if euclidean_distance[idx] > 40:
            classes.append('new_whale')
        else:
            classes.append(labels[idx])
    return classes

In [35]:
test_img = 'data/train/d9e958ed9.jpg'
feature = extractor(test_img, Encoder(model_path))
predict_label = top_5_classes(feature, train_features, labels)
print(predict_label)

['w_564a34b', 'w_05eb863', 'w_9b16b48', 'w_c3c0d22', 'new_whale']


In [36]:
def submmit(test_floder, model, output_file):
    files = natsorted(glob.glob(test_floder + '/*'))
    f = open(output_file, 'w')
    f.write('Image,Id\n')
    for file in tqdm(files):
        feature = extractor(file, model)
        top_5 = top_5_classes(feature, train_features, labels)
        img_name = file.split('/')[-1]
        f.write(' '.join([img_name+',', top_5[0], top_5[1], top_5[2], top_5[3],top_5[4], '\n']))
    f.close()

In [37]:
submmit('data/test', Encoder(model_path), 'submission.csv')

100%|██████████| 7960/7960 [22:22<00:00,  5.19it/s]
